In [ ]:
import sys
sys.path.insert(0, '../')

In [ ]:
from neural_map import NeuralMap, _plot
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

In [ ]:
df = pd.concat([pd.read_csv("datasets/table_219.csv"), pd.read_csv("datasets/table_220.csv"), pd.read_csv("datasets/table_221.csv")]).drop(['Sex', 'Delta 15 N (o/oo)', 'Delta 13 C (o/oo)', 'Comments'], axis=1).dropna().reset_index(drop=True)
l_columns = ['studyName', 'Sample Number', 'Species', 'Region', 'Island', 'Stage', 'Individual ID', 'Clutch Completion', 'Date Egg']
col = 'Species'

In [ ]:
scaler = MinMaxScaler()
data = scaler.fit_transform(df.drop(l_columns, axis=1).values)

data.shape

In [ ]:
som = NeuralMap(z=data.shape[1], x=10, y=10, metric='minkowski', p=5)

In [ ]:
som.train(data=data, verbosity=True, num_epochs=20, radius_decay_function='exponential')  # 
som.evaluate(data=data)

In [ ]:
som.plot_analysis(data, labels=df[col].values, min_cluster_size=3, show_empty_nodes=False)

In [ ]:
a = (1, 2)
type(a)

In [ ]:
som.plot_u_matrix(detailed=True, borders=True)

In [ ]:
actfreq, q, mean_distance = som.analyse(data)
umatrix, distance_matrix = som.unified_distance_matrix()
labels, probabilities, outlier_socer = som.hdbscan(min_cluster_size=3)

_plot.tiles(som.cart_coord, som.hexagonal, umatrix[..., -1], title='distancia')
_plot.tiles(som.cart_coord, som.hexagonal, actfreq, title='frecuencia de actvación')
_plot.tiles(som.cart_coord, som.hexagonal, q, title='error de cuantización')
_plot.tiles(som.cart_coord, som.hexagonal, mean_distance, title='distancia media')
_plot.tiles(som.cart_coord, som.hexagonal, probabilities, title = 'probabilidad de prtenencia')
_plot.tiles(som.cart_coord, som.hexagonal, outlier_socer, title = 'outiler score')

In [ ]:
clusters = 3

labels, centers = som.k_means(clusters)
_plot.tiles(som.cart_coord, som.hexagonal, labels, norm=False, title='Clustering con K-means. Valor de K = ' + str(clusters), labels=list(range(clusters)), cmap=plt.cm.get_cmap('hsv', clusters + 1))

labels, centers = som.k_medoids(clusters)
_plot.tiles(som.cart_coord, som.hexagonal, labels, norm=False, title='Clustering con K-medoids. Valor de K = ' + str(clusters), labels=list(range(clusters)), cmap=plt.cm.get_cmap('hsv', clusters + 1))

In [ ]:
som.plot_set_weight_vectors(cluster=0)
som.plot_node_weight_vector(node_index=(0, 0))